In [2]:
!pip -q install xgboost
import xgboost
print("xgboost:", xgboost.__version__)

xgboost: 3.1.3


In [4]:
!pip -q install xgboost joblib scikit-learn


In [5]:
# === Bloque 0: Setup (Colab) + verificación de entorno ===
from __future__ import annotations

import json
from pathlib import Path

import numpy as np
import pandas as pd
import sklearn
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error

try:
    import xgboost
    from xgboost import XGBRegressor
    xgb_version = xgboost.__version__
except Exception as e:
    xgb_version = f"NO DISPONIBLE ({e})"

# Semilla global para reproducibilidad
SEED: int = 42

# Directorios estándar del proyecto
REPORTS_DIR = Path("reports")
MODELS_DIR = Path("models")
REPORTS_DIR.mkdir(exist_ok=True)
MODELS_DIR.mkdir(exist_ok=True)

print("=== Versions ===")
print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("sklearn:", sklearn.__version__)
print("joblib:", joblib.__version__)
print("xgboost:", xgb_version)

print("\n=== Dirs ===")
print("reports dir:", REPORTS_DIR.resolve())
print("models dir:", MODELS_DIR.resolve())



=== Versions ===
numpy: 2.4.1
pandas: 2.2.2
sklearn: 1.8.0
joblib: 1.5.3
xgboost: 3.1.3

=== Dirs ===
reports dir: /content/reports
models dir: /content/models


In [7]:
# === Bloque 1: Carga de datos (consumos + sedes) ===
from google.colab import files

# Carga de datasets:
# - consumos_uptc.csv
# - sedes_uptc.csv
uploaded = files.upload()

df_cons = pd.read_csv("consumos_uptc.csv")
df_sedes = pd.read_csv("sedes_uptc.csv")

print("consumos shape:", df_cons.shape)
print("sedes shape:", df_sedes.shape)

print("\nColumnas consumos:\n", df_cons.columns.tolist())
print("\nColumnas sedes:\n", df_sedes.columns.tolist())

display(df_cons.head(3))
display(df_sedes.head(3))


Saving consumos_uptc.csv to consumos_uptc (1).csv
Saving sedes_uptc.csv to sedes_uptc (1).csv
consumos shape: (275387, 26)
sedes shape: (4, 17)

Columnas consumos:
 ['reading_id', 'timestamp', 'sede', 'sede_id', 'energia_total_kwh', 'energia_comedor_kwh', 'energia_salones_kwh', 'energia_laboratorios_kwh', 'energia_auditorios_kwh', 'energia_oficinas_kwh', 'potencia_total_kw', 'agua_litros', 'temperatura_exterior_c', 'ocupacion_pct', 'hora', 'dia_semana', 'dia_nombre', 'mes', 'trimestre', 'año', 'periodo_academico', 'es_fin_semana', 'es_festivo', 'es_semana_parciales', 'es_semana_finales', 'co2_kg']

Columnas sedes:
 ['sede', 'sede_id', 'nombre_completo', 'ciudad', 'area_m2', 'num_estudiantes', 'num_empleados', 'num_edificios', 'tiene_residencias', 'tiene_laboratorios_pesados', 'altitud_msnm', 'temp_promedio_c', 'pct_comedores', 'pct_salones', 'pct_laboratorios', 'pct_auditorios', 'pct_oficinas']


,reading_id,timestamp,sede,sede_id,energia_total_kwh,energia_comedor_kwh,energia_salones_kwh,energia_laboratorios_kwh,energia_auditorios_kwh,energia_oficinas_kwh,...,dia_nombre,mes,trimestre,año,periodo_academico,es_fin_semana,es_festivo,es_semana_parciales,es_semana_finales,co2_kg
0,1,2018-01-01 00:00:00,Chiquinquirá,UPTC_CHI,0.928,0.0452,0.1497,0.4334,0.0386,0.2613,...,Lunes,1,1,2018,vacaciones_fin,False,True,False,False,0.1877
1,2,2018-01-01 00:00:00,Duitama,UPTC_DUI,2.592,0.1129,0.2007,1.7993,0.0573,0.4217,...,Lunes,1,1,2018,vacaciones_fin,False,True,False,False,0.4490
2,3,2018-01-01 00:00:00,Sogamoso,UPTC_SOG,2.841,0.1395,0.2388,1.9638,NaN,0.4443,...,Lunes,1,1,2018,vacaciones_fin,False,True,False,False,0.6048


,sede,sede_id,nombre_completo,ciudad,area_m2,num_estudiantes,num_empleados,num_edificios,tiene_residencias,tiene_laboratorios_pesados,altitud_msnm,temp_promedio_c,pct_comedores,pct_salones,pct_laboratorios,pct_auditorios,pct_oficinas
0,Tunja,UPTC_TUN,Sede Central Tunja,Tunja,85000,18000,1200,25,True,True,2820,13,0.12,0.25,0.30,0.08,0.25
1,Duitama,UPTC_DUI,Sede Duitama,Duitama,35000,5500,350,12,False,True,2530,15,0.10,0.28,0.32,0.07,0.23
2,Sogamoso,UPTC_SOG,Sede Sogamoso,Sogamoso,40000,6000,400,14,False,True,2570,14,0.10,0.26,0.35,0.06,0.23


In [8]:
# === Bloque 2: Merge + saneo base ===

# 1) Definir columnas clave (ya confirmadas por tus columnas)
TIME_COL = "timestamp"
KEY_COL = "sede_id"
SEDE_COL = "sede"

# Target principal (baseline)
TARGET_COL = "energia_total_kwh"  # luego podrás cambiar a agua_litros o co2_kg

# 2) Parse timestamp y ordenar
df_cons = df_cons.copy()
df_cons[TIME_COL] = pd.to_datetime(df_cons[TIME_COL], errors="coerce")
df_cons = df_cons.dropna(subset=[TIME_COL]).sort_values(TIME_COL).reset_index(drop=True)

# 3) Merge (left join: consumos manda)
# Nota: evitamos duplicar la columna 'sede' si ya está en consumos
df_sedes_small = df_sedes.drop(columns=[SEDE_COL], errors="ignore")

df = df_cons.merge(df_sedes_small, on=KEY_COL, how="left")

print("df final shape:", df.shape)

# 4) Validaciones rápidas (fail fast)
missing_key = int(df[KEY_COL].isna().sum())
missing_target = int(df[TARGET_COL].isna().sum())
print("nulos en sede_id:", missing_key)
print(f"nulos en {TARGET_COL}:", missing_target)

print("rango temporal:", df[TIME_COL].min(), "->", df[TIME_COL].max())

# 5) Vista mínima
display(df[[TIME_COL, KEY_COL, SEDE_COL, TARGET_COL]].head(5))


df final shape: (275387, 41)
nulos en sede_id: 0
nulos en energia_total_kwh: 0
rango temporal: 2018-01-01 00:00:00 -> 2025-10-31 00:00:00


,timestamp,sede_id,sede,energia_total_kwh
0,2018-01-01 00:00:00,UPTC_CHI,Chiquinquirá,0.928
1,2018-01-01 00:00:00,UPTC_DUI,Duitama,2.592
2,2018-01-01 00:00:00,UPTC_SOG,Sogamoso,2.841
3,2018-01-01 00:00:00,UPTC_TUN,Tunja,1.505
4,2018-01-01 01:00:00,UPTC_CHI,Chiquinquirá,0.917


In [11]:
# === Bloque 3: Features + split temporal (por timestamp) + entrenamiento + export ===
# En este bloque entrenamos un modelo tabular para predecir el consumo de energía.
# Usamos variables temporales, contexto de sede y señales externas disponibles.
# La evaluación se hace con un corte temporal por timestamp para respetar la naturaleza de serie de tiempo.

df = df.copy()

# 1) Aseguramos variables temporales (si ya vienen, las reutilizamos; si no, las generamos).
if "hora" not in df.columns:
    df["hora"] = df[TIME_COL].dt.hour.astype("int16")
if "dia_semana" not in df.columns:
    df["dia_semana"] = df[TIME_COL].dt.dayofweek.astype("int16")
if "mes" not in df.columns:
    df["mes"] = df[TIME_COL].dt.month.astype("int16")
if "es_fin_semana" not in df.columns:
    df["es_fin_semana"] = (df["dia_semana"] >= 5).astype("int8")

# 2) Definimos variables: temporales + sede + señales externas (si existen).
base_features = ["hora", "dia_semana", "mes", "es_fin_semana"]
extra_candidates = ["temperatura_exterior_c", "ocupacion_pct", "es_festivo"]
extra_features = [c for c in extra_candidates if c in df.columns]

cat_features = ["sede_id"]
use_features = base_features + extra_features + cat_features

print("Features usadas:", use_features)

X = df[use_features].copy()
y = df[TARGET_COL].copy()

# Convertimos sede_id a one-hot para capturar patrones por sede en un modelo tabular.
X = pd.get_dummies(X, columns=cat_features, drop_first=False)

# 3) Corte temporal por timestamp (85% del tiempo para entrenamiento).
cutoff = df[TIME_COL].quantile(0.85)
train_mask = df[TIME_COL] < cutoff
test_mask = ~train_mask

X_train, X_test = X.loc[train_mask], X.loc[test_mask]
y_train, y_test = y.loc[train_mask], y.loc[test_mask]

print("Cutoff timestamp:", cutoff)
print("Train:", X_train.shape, "Test:", X_test.shape)
print("Train rango:", df.loc[train_mask, TIME_COL].min(), "->", df.loc[train_mask, TIME_COL].max())
print("Test rango:", df.loc[test_mask, TIME_COL].min(), "->", df.loc[test_mask, TIME_COL].max())

# 4) Entrenamiento del modelo tabular (XGBoost).
model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    random_state=SEED,
    n_jobs=-1,
)
model.fit(X_train, y_train)

# 5) Predicción + métricas.
yhat_test = model.predict(X_test)

mae = float(mean_absolute_error(y_test, yhat_test))
mse = float(mean_squared_error(y_test, yhat_test))
rmse = float(np.sqrt(mse))

metrics = {
    "target": TARGET_COL,
    "time_col": TIME_COL,
    "split_method": "timestamp_quantile_0.85",
    "cutoff_timestamp": str(cutoff),
    "seed": SEED,
    "mae": mae,
    "rmse": rmse,
    "features_used": use_features,
    "rows_total": int(len(df)),
    "rows_train": int(train_mask.sum()),
    "rows_test": int(test_mask.sum()),
}

print("MAE:", mae)
print("RMSE:", rmse)

# 6) Exportamos artefactos estándar (modelo + reportes).
joblib.dump(model, MODELS_DIR / "model_energy.joblib")

(REPORTS_DIR / "metrics_energy.json").write_text(
    json.dumps(metrics, indent=2, ensure_ascii=False),
    encoding="utf-8"
)

df_test = df.loc[test_mask, [TIME_COL, "sede_id", "sede"]].copy()
df_test["y"] = y_test.values
df_test["yhat"] = yhat_test

pred_vs_real = df_test[[TIME_COL, "sede_id", "sede", "y", "yhat"]].copy()
pred_vs_real.to_csv(REPORTS_DIR / "pred_vs_real_energy.csv", index=False)

print("Exportados:")
print("-", REPORTS_DIR / "metrics_energy.json")
print("-", REPORTS_DIR / "pred_vs_real_energy.csv")
print("-", MODELS_DIR / "model_energy.joblib")

pred_vs_real.head(5)


Features usadas: ['hora', 'dia_semana', 'mes', 'es_fin_semana', 'temperatura_exterior_c', 'ocupacion_pct', 'es_festivo', 'sede_id']
Cutoff timestamp: 2024-08-27 22:06:00
Train: (234079, 19) Test: (41308, 19)
Train rango: 2018-01-01 00:00:00 -> 2024-08-27 22:00:00
Test rango: 2024-08-27 23:00:00 -> 2025-10-31 00:00:00
MAE: 1.0823116335952911
RMSE: 4.312126233110544
Exportados:
- reports/metrics_energy.json
- reports/pred_vs_real_energy.csv
- models/model_energy.joblib


,timestamp,sede_id,sede,y,yhat
234079,2024-08-27 23:00:00,UPTC_CHI,Chiquinquirá,1.463,1.461215
234080,2024-08-27 23:00:00,UPTC_DUI,Duitama,5.898,5.490296
234081,2024-08-27 23:00:00,UPTC_SOG,Sogamoso,5.801,5.048595
234082,2024-08-27 23:00:00,UPTC_TUN,Tunja,2.294,2.238086
234083,2024-08-28 00:00:00,UPTC_CHI,Chiquinquirá,1.463,1.513348


In [12]:
# === Bloque 4: Anomalías por residuales (umbral p99) ===
# Detectamos anomalías con base en el error del modelo: |real - predicho|.
# Usamos p99 para reportar los casos más extremos (alta severidad).

anoms_df = pred_vs_real.copy()
anoms_df["residual"] = anoms_df["y"] - anoms_df["yhat"]
anoms_df["abs_residual"] = np.abs(anoms_df["residual"])

threshold = float(anoms_df["abs_residual"].quantile(0.99))
anoms_df["threshold_p99"] = threshold
anoms_df["is_anomaly"] = anoms_df["abs_residual"] >= threshold

anomalies = (
    anoms_df[anoms_df["is_anomaly"]]
    .sort_values("abs_residual", ascending=False)
    .reset_index(drop=True)
)

out_path = REPORTS_DIR / "anomalies_energy.csv"
anomalies.to_csv(out_path, index=False)

print("Umbral p99 (abs residual):", threshold)
print("Total anomalías:", anomalies.shape[0])
print("Exportado:", out_path)

anomalies.head(10)


Umbral p99 (abs residual): 9.492739099744208
Total anomalías: 414
Exportado: reports/anomalies_energy.csv


,timestamp,sede_id,sede,y,yhat,residual,abs_residual,threshold_p99,is_anomaly
0,2025-09-01 11:00:00,UPTC_SOG,Sogamoso,187.362699,22.108442,165.254257,165.254257,9.492739,True
1,2024-09-27 08:00:00,UPTC_DUI,Duitama,183.635684,24.173157,159.462527,159.462527,9.492739,True
2,2025-10-28 11:00:00,UPTC_SOG,Sogamoso,169.340477,18.801157,150.539320,150.539320,9.492739,True
3,2025-09-22 14:00:00,UPTC_DUI,Duitama,171.442714,21.945251,149.497462,149.497462,9.492739,True
4,2024-09-24 11:00:00,UPTC_SOG,Sogamoso,171.677879,22.852081,148.825798,148.825798,9.492739,True
5,2025-04-08 11:00:00,UPTC_SOG,Sogamoso,168.136145,22.115608,146.020537,146.020537,9.492739,True
6,2025-04-09 14:00:00,UPTC_DUI,Duitama,159.983744,15.662533,144.321211,144.321211,9.492739,True
7,2025-10-21 07:00:00,UPTC_SOG,Sogamoso,162.727341,18.677982,144.049358,144.049358,9.492739,True
8,2024-12-05 09:00:00,UPTC_SOG,Sogamoso,151.681692,16.909081,134.772612,134.772612,9.492739,True
9,2024-10-18 08:00:00,UPTC_SOG,Sogamoso,151.040332,20.292356,130.747976,130.747976,9.492739,True


In [13]:
# === Bloque 5: Recomendaciones basadas en reglas (accionables) ===
# Generamos recomendaciones simples y defendibles a partir de anomalías:
# - Si el consumo real supera mucho la predicción: posible sobreconsumo / equipos encendidos.
# - Si ocurre fuera de horario o fin de semana: priorizar revisión de operación.
# - Reportamos acciones sugeridas por severidad.

recs = anomalies.copy()

# Severidad por cuantiles del abs_residual
q90 = float(recs["abs_residual"].quantile(0.90))
q97 = float(recs["abs_residual"].quantile(0.97))

def severity(x: float) -> str:
    if x >= q97:
        return "alta"
    if x >= q90:
        return "media"
    return "baja"

recs["severity"] = recs["abs_residual"].apply(severity)

# Señales temporales (si no existen, las inferimos de timestamp)
recs[TIME_COL] = pd.to_datetime(recs[TIME_COL], errors="coerce")
recs["hora"] = recs[TIME_COL].dt.hour
recs["dia_semana"] = recs[TIME_COL].dt.dayofweek
recs["es_fin_semana"] = (recs["dia_semana"] >= 5)

# Regla de "fuera de horario" (ajustable)
recs["fuera_horario"] = (recs["hora"] < 6) | (recs["hora"] > 20)

# Recomendación principal
def make_action(row) -> str:
    if row["severity"] == "alta" and (row["fuera_horario"] or row["es_fin_semana"]):
        return "Revisar equipos/iluminación encendidos fuera de horario; validar rutinas de apagado y automatización."
    if row["severity"] in {"alta", "media"} and row["fuera_horario"]:
        return "Auditar consumo nocturno: HVAC, laboratorios y cargas base; programar apagados y alertas."
    if row["severity"] in {"alta", "media"} and row["es_fin_semana"]:
        return "Verificar uso en fin de semana; validar actividades, horarios especiales o consumos anómalos."
    if row["severity"] == "alta":
        return "Inspección técnica prioritaria: medir cargas por circuito/área; revisar equipos de alto consumo."
    return "Monitorear patrón; si se repite, revisar operación y registrar causa."

recs["action"] = recs.apply(make_action, axis=1)

# Compactamos columnas clave para el reporte final
recommendations = recs[[
    TIME_COL, "sede_id", "sede", "y", "yhat", "abs_residual", "severity",
    "es_fin_semana", "fuera_horario", "action"
]].copy()

# Ordenamos por severidad y magnitud
sev_order = {"alta": 0, "media": 1, "baja": 2}
recommendations["sev_rank"] = recommendations["severity"].map(sev_order)
recommendations = recommendations.sort_values(["sev_rank", "abs_residual"], ascending=[True, False]).drop(columns=["sev_rank"])

out_path = REPORTS_DIR / "recommendations_energy.csv"
recommendations.to_csv(out_path, index=False)

print("Exportado:", out_path)
recommendations.head(15)


Exportado: reports/recommendations_energy.csv


,timestamp,sede_id,sede,y,yhat,abs_residual,severity,es_fin_semana,fuera_horario,action
0,2025-09-01 11:00:00,UPTC_SOG,Sogamoso,187.362699,22.108442,165.254257,alta,False,False,Inspección técnica prioritaria: medir cargas p...
1,2024-09-27 08:00:00,UPTC_DUI,Duitama,183.635684,24.173157,159.462527,alta,False,False,Inspección técnica prioritaria: medir cargas p...
2,2025-10-28 11:00:00,UPTC_SOG,Sogamoso,169.340477,18.801157,150.539320,alta,False,False,Inspección técnica prioritaria: medir cargas p...
3,2025-09-22 14:00:00,UPTC_DUI,Duitama,171.442714,21.945251,149.497462,alta,False,False,Inspección técnica prioritaria: medir cargas p...
4,2024-09-24 11:00:00,UPTC_SOG,Sogamoso,171.677879,22.852081,148.825798,alta,False,False,Inspección técnica prioritaria: medir cargas p...
5,2025-04-08 11:00:00,UPTC_SOG,Sogamoso,168.136145,22.115608,146.020537,alta,False,False,Inspección técnica prioritaria: medir cargas p...
6,2025-04-09 14:00:00,UPTC_DUI,Duitama,159.983744,15.662533,144.321211,alta,False,False,Inspección técnica prioritaria: medir cargas p...
7,2025-10-21 07:00:00,UPTC_SOG,Sogamoso,162.727341,18.677982,144.049358,alta,False,False,Inspección técnica prioritaria: medir cargas p...
8,2024-12-05 09:00:00,UPTC_SOG,Sogamoso,151.681692,16.909081,134.772612,alta,False,False,Inspección técnica prioritaria: medir cargas p...
9,2024-10-18 08:00:00,UPTC_SOG,Sogamoso,151.040332,20.292356,130.747976,alta,False,False,Inspección técnica prioritaria: medir cargas p...


In [14]:
# === Bloque 6: Forecast futuro (24h y 7d) por sede_id + export ===
# Aquí generamos filas futuras y predecimos con el modelo ya entrenado.
# Para variables externas (temperatura/ocupación), usamos promedios históricos por (sede_id, hora).
# Para es_festivo, si no tenemos calendario futuro, usamos 0 como supuesto (documentable).

import numpy as np
import pandas as pd
import joblib
from pathlib import Path

REPORTS_DIR = Path("reports")
MODELS_DIR = Path("models")

# 1) Cargar modelo
model = joblib.load(MODELS_DIR / "model_energy.joblib")

# 2) Definir horizonte
H24 = 24
H7D = 24 * 7

# 3) Preparar base: sedes y timestamp de inicio
df[TIME_COL] = pd.to_datetime(df[TIME_COL], errors="coerce")
last_ts = df[TIME_COL].max()
start_ts = (last_ts + pd.Timedelta(hours=1)).floor("H")

sede_ids = sorted(df["sede_id"].dropna().unique().tolist())

# 4) Función: construir futuro dataframe (h horas)
def build_future(h_hours: int) -> pd.DataFrame:
    future_ts = pd.date_range(start=start_ts, periods=h_hours, freq="H")
    fut = pd.DataFrame(
        [(ts, sid) for ts in future_ts for sid in sede_ids],
        columns=[TIME_COL, "sede_id"]
    )

    # Variables temporales
    fut["hora"] = fut[TIME_COL].dt.hour.astype("int16")
    fut["dia_semana"] = fut[TIME_COL].dt.dayofweek.astype("int16")
    fut["mes"] = fut[TIME_COL].dt.month.astype("int16")
    fut["es_fin_semana"] = (fut["dia_semana"] >= 5).astype("int8")

    # Variables externas: promedio histórico por (sede_id, hora) si existen
    if "temperatura_exterior_c" in df.columns:
        temp_map = (
            df.groupby(["sede_id", "hora"])["temperatura_exterior_c"]
              .mean()
              .reset_index()
        )
        fut = fut.merge(temp_map, on=["sede_id", "hora"], how="left")
        # fallback: promedio global
        fut["temperatura_exterior_c"] = fut["temperatura_exterior_c"].fillna(df["temperatura_exterior_c"].mean())

    if "ocupacion_pct" in df.columns:
        occ_map = (
            df.groupby(["sede_id", "hora"])["ocupacion_pct"]
              .mean()
              .reset_index()
        )
        fut = fut.merge(occ_map, on=["sede_id", "hora"], how="left")
        fut["ocupacion_pct"] = fut["ocupacion_pct"].fillna(df["ocupacion_pct"].mean())

    # Festivos: si no hay calendario futuro, asumimos 0
    if "es_festivo" in df.columns:
        fut["es_festivo"] = 0

    # Nombre de sede (para demo)
    if "sede" in df.columns:
        sede_name = df[["sede_id", "sede"]].drop_duplicates()
        fut = fut.merge(sede_name, on="sede_id", how="left")

    return fut

# 5) Preparar columnas de entrada EXACTAS del modelo
#    (xgboost suele exponer feature_names_in_ o booster.feature_names)
def get_model_feature_names(m):
    if hasattr(m, "feature_names_in_"):
        return list(m.feature_names_in_)
    try:
        return list(m.get_booster().feature_names)
    except Exception:
        return None

model_features = get_model_feature_names(model)
if model_features is None:
    raise RuntimeError("No pude obtener nombres de features del modelo. Re-entrena guardando X_train.columns.")

def predict_future(fut: pd.DataFrame) -> pd.DataFrame:
    # Selección de features (las mismas usadas en entrenamiento)
    base_features = ["hora", "dia_semana", "mes", "es_fin_semana"]
    extra_candidates = ["temperatura_exterior_c", "ocupacion_pct", "es_festivo"]
    extra_features = [c for c in extra_candidates if c in fut.columns]
    use_features = base_features + extra_features + ["sede_id"]

    Xf = fut[use_features].copy()
    Xf = pd.get_dummies(Xf, columns=["sede_id"], drop_first=False)

    # Alinear columnas al modelo
    for col in model_features:
        if col not in Xf.columns:
            Xf[col] = 0
    Xf = Xf[model_features]

    yhat = model.predict(Xf)
    out = fut[[TIME_COL, "sede_id"]].copy()
    if "sede" in fut.columns:
        out["sede"] = fut["sede"]
    out["horizon_hours"] = int(len(fut[TIME_COL].unique()))
    out["yhat"] = yhat
    return out

# 6) Forecast 24h
fut24 = build_future(H24)
fc24 = predict_future(fut24)
fc24_path = REPORTS_DIR / "forecast_24h_energy.csv"
fc24.to_csv(fc24_path, index=False)

# 7) Forecast 7d
fut7d = build_future(H7D)
fc7d = predict_future(fut7d)
fc7d_path = REPORTS_DIR / "forecast_7d_energy.csv"
fc7d.to_csv(fc7d_path, index=False)

print("Exportados:")
print("-", fc24_path)
print("-", fc7d_path)

fc24.head(10)


/tmp/ipython-input-1972726138.py:24: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  start_ts = (last_ts + pd.Timedelta(hours=1)).floor("H")
/tmp/ipython-input-1972726138.py:30: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_ts = pd.date_range(start=start_ts, periods=h_hours, freq="H")
/tmp/ipython-input-1972726138.py:30: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_ts = pd.date_range(start=start_ts, periods=h_hours, freq="H")


Exportados:
- reports/forecast_24h_energy.csv
- reports/forecast_7d_energy.csv


,timestamp,sede_id,sede,horizon_hours,yhat
0,2025-10-31 01:00:00,UPTC-CHI,Chiquinquirá,24,1.352732
1,2025-10-31 01:00:00,UPTC-DUI,Duitama,24,2.674185
2,2025-10-31 01:00:00,UPTC-SOG,Sogamoso,24,3.617930
3,2025-10-31 01:00:00,UPTC-TUN,Tunja,24,1.244031
4,2025-10-31 01:00:00,UPTC_CHI,Chiquinquirá,24,0.667247
5,2025-10-31 01:00:00,UPTC_DUI,Duitama,24,2.036629
6,2025-10-31 01:00:00,UPTC_SOG,Sogamoso,24,2.210204
7,2025-10-31 01:00:00,UPTC_TUN,Tunja,24,1.042236
8,2025-10-31 01:00:00,uptc_chi,Chiquinquirá,24,0.957604
9,2025-10-31 01:00:00,uptc_dui,Duitama,24,2.124490
